In [72]:
import pandas as pd
import re
tokens = open(".env", "r").read()

# Ouraring exploration

In [65]:
from sys import api_version
import ouraring
from ouraring import Configuration

# https://cloud.ouraring.com/personal-access-tokens
ouraring_token = re.findall(r"OURARING_TOKEN=\"(.*)\"", tokens)[0]
conf = Configuration(
    host="https://api.ouraring.com/v2",
    api_key=ouraring_token,
    access_token=ouraring_token,
)
ouraring_client = ouraring.ApiClient(
    header_name="Authorization",
    header_value="Bearer " + ouraring_token,
    configuration=conf,
)
from ouraring.api.daily_sleep_api import DailySleepApi

daily_sleep_api = DailySleepApi(ouraring_client)

In [ ]:
import requests

url = "https://api.ouraring.com/v2/usercollection/daily_readiness"
params = {"start_date": "2021-11-01", "end_date": "2023-12-01"}
headers = {"Authorization": "Bearer " + ouraring_token}

daily_readiness_df = pd.json_normalize(
    requests.request("GET", url, headers=headers, params=params).json()["data"],
)
# replace all columns starting with "contributors." by "daily_readiness_"
daily_readiness_df.columns = [
    re.sub(r"^contributors\.", "daily_readiness_", col)
    for col in daily_readiness_df.columns
]
# replace "score" by "daily_readiness_score"
daily_readiness_df.columns = [
    re.sub(r"^score$", "daily_readiness_score", col)
    for col in daily_readiness_df.columns
]
url = "https://api.ouraring.com/v2/usercollection/heartrate"
heartrate_df = pd.json_normalize(
    requests.request("GET", url, headers=headers, params=params).json()["data"],
)

url = "https://api.ouraring.com/v2/usercollection/daily_sleep"
daily_sleep_df = pd.json_normalize(
    requests.request("GET", url, headers=headers, params=params).json()["data"],
)
# replace all columns starting with "contributors." by "daily_sleep_"
daily_sleep_df.columns = [
    re.sub(r"^contributors\.", "daily_sleep_", col) for col in daily_sleep_df.columns
]
# replace "score" by "daily_sleep_score"
daily_sleep_df.columns = [
    re.sub(r"^score$", "daily_sleep_score", col) for col in daily_sleep_df.columns
]

url = "https://api.ouraring.com/v2/usercollection/daily_activity"
daily_activity_df = pd.json_normalize(
    requests.request("GET", url, headers=headers, params=params).json()["data"],
)
# replace all columns starting with "contributors." by "daily_activity_"
daily_activity_df.columns = [
    re.sub(r"^contributors\.", "daily_activity_", col)
    for col in daily_activity_df.columns
]
# replace all columns starting with "met." by "daily_activity_"
daily_activity_df.columns = [
    re.sub(r"^met\.", "daily_activity_", col) for col in daily_activity_df.columns
]
# replace "score" by "daily_activity_score"
daily_activity_df.columns = [
    re.sub(r"^score$", "daily_activity_score", col) for col in daily_activity_df.columns
]

# merge all dataframes on nearest "timestamp" column
# type 'datetime64[ns]'
daily_activity_df["date"] = pd.to_datetime(daily_activity_df.timestamp).tz_localize(None)
daily_sleep_df["date"] = pd.to_datetime(daily_sleep_df.timestamp).tz_localize(None)
daily_readiness_df["date"] = pd.to_datetime(daily_readiness_df.timestamp).tz_localize(None)
daily_activity_df.sort_values("date", inplace=True)
daily_sleep_df.sort_values("date", inplace=True)
daily_readiness_df.sort_values("date", inplace=True)
df = pd.merge_asof(
    daily_activity_df,
    daily_sleep_df,
    on="date",
    by="date",
    direction="nearest",
    tolerance=pd.Timedelta("1 day"),
)
df = pd.merge_asof(
    df,
    daily_readiness_df,
    on="date",
    by="date",
    direction="nearest",
    tolerance=pd.Timedelta("1 day"),
)
df

In [88]:
from ouraring.apis import DailySleepApi
from ouraring import ApiClient

DailySleepApi(
    ApiClient(
        configuration=Configuration(
            host="https://api.ouraring.com",
            api_key=ouraring_token,
            access_token=ouraring_token,
        ),
        header_name="Authorization",
        header_value="Bearer " + ouraring_token,
    )
).daily_sleep_route_daily_sleep_get(
    start_date="2021-11-01",
    end_date="2023-12-01",
)


TypeError: _from_openapi_data() missing 3 required positional arguments: 'contributors', 'day', and 'timestamp'

In [ ]:
params = {"start_date": "2022-11-01", "end_date": "2023-12-01"}
import json
daily_sleep_api.daily_sleep_route_daily_sleep_get()

# Timingapp exploration

https://timingapp.com

In [4]:
import timingapp

In [5]:
timingapp_token = re.findall(r"TIMINGAPP_TOKEN=\"(.*)\"", tokens)[0]

In [7]:
timingapp_client = timingapp.ApiClient(header_name="Authorization", header_value="Bearer " + timingapp_token)

In [ ]:
from timingapp.api.reports_api import ReportsApi
report_api = ReportsApi(timingapp_client)
pd.DataFrame(report_api.api_v1_report_get()["data"]).sort_values("duration", ascending=False)